This code finds uncorrelated, high volume USDT pairs to trade in Binance

This code is helpful for finding uncorrolated XXXUSDT pairs to trade on Binance Exchange

In [7]:
from binance.client import Client
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import IPython.display as display

# Read API keys from the keys file
keys_file_path = 'C:/Users/PC/Desktop/keys.txt'

with open(keys_file_path, 'r') as f:
    lines = f.readlines()

api_key_line = lines[0]
api_secret_line = lines[1]

api_key = api_key_line.split('=')[1].strip().strip("'").strip('"')
api_secret = api_secret_line.split('=')[1].strip().strip("'").strip('"')

client = Client(api_key, api_secret)

# Pandas uyarılarını kapat
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.mode.use_inf_as_na = True

# XXXUSDT paritelerini bul ve ilk 5 pariteyi seç
tickers = client.get_all_tickers()
usdt_pairs = [ticker['symbol'] for ticker in tickers if ticker['symbol'].endswith('USDT') and ticker['symbol'] != 'USDTUSDT']
#usdt_pairs = [ticker['symbol'] for ticker in tickers if ticker['symbol'].endswith('USDT') and ticker['symbol'] != 'USDTUSDT'][:5]
print("Selected Pairs:", usdt_pairs)

# Veri toplama fonksiyonu
def fetch_klines(symbol, interval, start_date, end_date):
    return client.get_historical_klines(symbol, interval, start_date.strftime('%d %b %Y %H:%M:%S'), end_date.strftime('%d %b %Y %H:%M:%S'))

# Son 1 yıl, son 3 ay ve son 1 ay için veri periyotlarını belirle
end_date = datetime.now()
start_date_1y = end_date - timedelta(days=365)
start_date_3m = end_date - timedelta(days=90)
start_date_1m = end_date - timedelta(days=30)

# Kapanış fiyatları için veri toplama
klines_1y = {pair: fetch_klines(pair, Client.KLINE_INTERVAL_1DAY, start_date_1y, end_date) for pair in usdt_pairs}
klines_3m = {pair: fetch_klines(pair, Client.KLINE_INTERVAL_1DAY, start_date_3m, end_date) for pair in usdt_pairs}
klines_1m = {pair: fetch_klines(pair, Client.KLINE_INTERVAL_1DAY, start_date_1m, end_date) for pair in usdt_pairs}

# Fiyat verilerini DataFrame'e dönüştürme
def klines_to_df(klines):
    return pd.DataFrame({
        'open_time': [datetime.fromtimestamp(entry[0] / 1000) for entry in klines],
        'close': [float(entry[4]) for entry in klines]
    })

dfs_1y = {pair: klines_to_df(klines) for pair, klines in klines_1y.items()}
dfs_3m = {pair: klines_to_df(klines) for pair, klines in klines_3m.items()}
dfs_1m = {pair: klines_to_df(klines) for pair, klines in klines_1m.items()}

# Kapanış fiyatlarını birleştir
def merge_close_prices(dfs):
    close_prices = pd.concat({pair: df['close'] for pair, df in dfs.items()}, axis=1)
    close_prices.index = dfs[usdt_pairs[0]]['open_time']
    return close_prices

close_prices_1y = merge_close_prices(dfs_1y)
close_prices_3m = merge_close_prices(dfs_3m)
close_prices_1m = merge_close_prices(dfs_1m)

# Korelasyonları hesapla
correlation_matrix_1y = close_prices_1y.corr().round(2)
correlation_matrix_3m = close_prices_3m.corr().round(2)
correlation_matrix_1m = close_prices_1m.corr().round(2)

# NaN değerleri içeren sütunları ve satırları çıkar
correlation_matrix_1y = correlation_matrix_1y.dropna(axis=0, how='all').dropna(axis=1, how='all')
correlation_matrix_3m = correlation_matrix_3m.dropna(axis=0, how='all').dropna(axis=1, how='all')
correlation_matrix_1m = correlation_matrix_1m.dropna(axis=0, how='all').dropna(axis=1, how='all')

# Korelasyonu corr_threshold'dan az olan pariteleri filtreleme
corr_threshold = 1  # Bu değeri parametrik olarak değiştirebilirsiniz
filtered_corr_1y = correlation_matrix_1y[correlation_matrix_1y <= corr_threshold]
filtered_corr_3m = correlation_matrix_3m[correlation_matrix_3m <= corr_threshold]
filtered_corr_1m = correlation_matrix_1m[correlation_matrix_1m <= corr_threshold]

# Sadece NaN olmayan değerleri içeren DataFrame'ler oluşturma
filtered_corr_1y = filtered_corr_1y.dropna(how='all', axis=0).dropna(how='all', axis=1)
filtered_corr_3m = filtered_corr_3m.dropna(how='all', axis=0).dropna(how='all', axis=1)
filtered_corr_1m = filtered_corr_1m.dropna(how='all', axis=0).dropna(how='all', axis=1)

# Pop-up tablo gösterme
# print(f"1-Year Correlation Matrix (Filtered < {corr_threshold}):")
# display.display(filtered_corr_1y)

# print(f"3-Month Correlation Matrix (Filtered < {corr_threshold}):")
# display.display(filtered_corr_3m)

# print(f"1-Month Correlation Matrix (Filtered < {corr_threshold}):")
# display.display(filtered_corr_1m)

# Excel dosyasına yazma
output_file = 'C:/Users/PC/Desktop/correlation_output.xlsx'  # Çıktı dosyasının yolu
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    filtered_corr_1m.to_excel(writer, sheet_name='1-Month Correlation')
    filtered_corr_3m.to_excel(writer, sheet_name='3-Month Correlation')
    filtered_corr_1y.to_excel(writer, sheet_name='1-Year Correlation')

print(f"Korelasyon verileri {output_file} dosyasına yazıldı.")


C:\Users\PC\AppData\Local\Temp\ipykernel_2548\1970768709.py:23: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.options.mode.use_inf_as_na = True


Selected Pairs: ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'BCCUSDT', 'NEOUSDT', 'LTCUSDT', 'QTUMUSDT', 'ADAUSDT', 'XRPUSDT', 'EOSUSDT', 'TUSDUSDT', 'IOTAUSDT', 'XLMUSDT', 'ONTUSDT', 'TRXUSDT', 'ETCUSDT', 'ICXUSDT', 'VENUSDT', 'NULSUSDT', 'VETUSDT', 'PAXUSDT', 'BCHABCUSDT', 'BCHSVUSDT', 'USDCUSDT', 'LINKUSDT', 'WAVESUSDT', 'BTTUSDT', 'USDSUSDT', 'ONGUSDT', 'HOTUSDT', 'ZILUSDT', 'ZRXUSDT', 'FETUSDT', 'BATUSDT', 'XMRUSDT', 'ZECUSDT', 'IOSTUSDT', 'CELRUSDT', 'DASHUSDT', 'NANOUSDT', 'OMGUSDT', 'THETAUSDT', 'ENJUSDT', 'MITHUSDT', 'MATICUSDT', 'ATOMUSDT', 'TFUELUSDT', 'ONEUSDT', 'FTMUSDT', 'ALGOUSDT', 'USDSBUSDT', 'GTOUSDT', 'ERDUSDT', 'DOGEUSDT', 'DUSKUSDT', 'ANKRUSDT', 'WINUSDT', 'COSUSDT', 'NPXSUSDT', 'COCOSUSDT', 'MTLUSDT', 'TOMOUSDT', 'PERLUSDT', 'DENTUSDT', 'MFTUSDT', 'KEYUSDT', 'STORMUSDT', 'DOCKUSDT', 'WANUSDT', 'FUNUSDT', 'CVCUSDT', 'CHZUSDT', 'BANDUSDT', 'BUSDUSDT', 'BEAMUSDT', 'XTZUSDT', 'RENUSDT', 'RVNUSDT', 'HCUSDT', 'HBARUSDT', 'NKNUSDT', 'STXUSDT', 'KAVAUSDT', 'ARPAUSDT', 'IOTXUSDT', '